In [1]:
import os

In [4]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [21]:
load_dotenv()

True

In [16]:
model = ChatGroq(model='qwen/qwen3-32b' )

In [17]:
model.invoke('the sky is').content

'<think>\nOkay, the user wrote "the sky is". They might be trying to start a sentence or a question. Let me think about possible continuations. Maybe they want to describe the sky in a certain way, like "the sky is blue" or "the sky is cloudy". Alternatively, they could be asking for information, such as "the sky is what color?" or "the sky is made of what?".\n\nSince the input is very short, I should consider if they need help completing a thought or if they have a specific question in mind. They might be referring to a common phrase or a riddle. For example, "The sky is the limit" or "The sky is falling". \n\nI should also check if there\'s any context from previous messages, but since this is the first message, there\'s nothing to reference. The best approach is to ask for clarification. I can offer examples to prompt them to elaborate on what they\'re thinking. That way, I can provide a more accurate and helpful response.\n\nSo, I\'ll respond by asking what they were going to say n

In [19]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [25]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [27]:
embedding = embedding_model.embed_query('the sky is')

In [29]:
len(embedding)

768